In [ ]:
%pylab inline
import glob
import pandas as pd
import os
import datetime as dt
import seaborn as sns

from glab_common import utils
data_path = '/mnt/cube/RawData/Zog/'

subjects = (
            'B1100',
            'B1069'
            )

## plot performance over past two weeks in calendar

In [ ]:
def plot_performance_calendar(subj,df,num_days=7):
    today = dt.datetime.now()
    data_to_analyze = df[(df.response<>'none')&(df.type_=='normal')] #still a dataframe (a filtered one)
    blocked = data_to_analyze.groupby([lambda x: (today.date()-x.date()).days, lambda x: x.hour])
        #blocked is a grouped data structure
    
    aggregated = blocked.agg({'correct': lambda x: np.mean(x.astype(float)),
                              'reward': lambda x: np.sum(x.astype(float)),
                              'response': lambda x: np.mean((x=='L').astype(float)),
                              'type_': lambda x: np.sum((x=='normal').astype(float)),
                                })
    days_past = np.arange(num_days)
    hour_of_day = np.arange(24)
    
    skip = num_days / 7
    if skip < 1: skip = 1
        
    dd = []
    day_index = [d for d in days_past if (d % skip) > 0]
    for d in days_past[::skip]:
        datetime = today-dt.timedelta(days=num_days-1-d)
        if (len(dd) < 1) or (datetime.day<(skip+1)):
            dd.append(str(datetime.day)+'\n'+str(datetime.strftime('%b')))
        else:
            dd.append(str(datetime.day))
    
    hh = ['6pm','noon','6am']
            
    f, ax = plt.subplots(1,3,sharex=True,sharey=True,figsize=(16.0, 4.0))
    # first, display the trials
    cmap = plt.get_cmap('Oranges')
    num_trials = np.zeros((len(days_past),len(hour_of_day)),np.float_)
    for hour in hour_of_day:
        for day in days_past:
            try:
                num_trials[day,hour] = aggregated['type_'][day,hour]
            except KeyError:
                num_trials[day,hour] = 0.0
    ax[0].pcolor(np.rot90(np.fliplr(num_trials),k=3),cmap=cmap,vmin=0, vmax=150)
    ax[0].axis('tight')
    
    ax[0].set_title(subj+': Trials per hour')
    ax[0].set_xticks(days_past[::skip]+0.5)
    ax[0].set_xticklabels(dd)
    ax[0].set_yticks([6,12,18])
    ax[0].set_yticklabels(hh)
   
        
    cmap = plt.get_cmap('RdYlBu')
    cmap.set_bad(color='Grey')
    
    correct = np.zeros((len(days_past),len(hour_of_day)),np.float_)
    for hour in hour_of_day:
        for day in days_past:
            try:
                correct[day,hour] = aggregated['correct'][day,hour]
            except KeyError:
                correct[day,hour] = np.nan
    correct = np.ma.masked_invalid(correct)
    ax[1].pcolormesh(np.rot90(np.fliplr(correct),k=3),cmap=cmap,vmin=0, vmax=1)
    ax[1].set_title('Accuracy')
    
    feeds = np.zeros((len(days_past),len(hour_of_day)),np.float_)
    for hour in hour_of_day:
        for day in days_past:
            try:
                feeds[day,hour] = aggregated['reward'][day,hour]
            except KeyError:
                feeds[day,hour] = 0.0
    feeds = np.ma.masked_invalid(feeds)
    ax[2].pcolormesh(np.rot90(np.fliplr(feeds),k=3),cmap=plt.get_cmap('BuGn'),vmin=0, vmax=25)
    ax[2].set_title('Feeds')
    
    return f, ax

#do the plot
behav_data = utils.load_data_pandas(subjects,data_path)

for subj,data in behav_data.iteritems():
    plot_performance_calendar(subj,data,num_days=20)    

## Plot percent correct by block with confidence bounds

In [ ]:
behav_data = utils.load_data_pandas(subjects,data_path)
figsize(16.0, 16.0)

n_birds = len(behav_data.keys())
fig, ax = plt.subplots(nrows=n_birds,ncols=1)

for s, subj in enumerate(subjects):
    df = behav_data[subj]
    data_to_analyze = df[(df.response<>'none')&(df.type_=='normal')].sort()
    data_to_analyze = data_to_analyze[-10000:]
    data_to_analyze = data_to_analyze.reset_index()
    
    block_size = 100
    
    blocks = data_to_analyze.index / block_size
    blocked = data_to_analyze.groupby(blocks)
    
    sca(ax[s])
    correct = blocked['correct'].agg({'correct': lambda x: np.mean(x.astype(float))})
    correct.plot(ax=ax[s])
    
    ci = utils.binomial_ci(0.5*block_size,block_size)
    axhspan(ci[0],ci[1],color='grey',alpha=0.5,axes=ax[s])
    axhline(0.5,linestyle=':',axes=ax[s],color='grey') 
    
    ylim([0,1.0])
    title(subj)
    ylabel('proportion')
    
    bias = blocked['response'].agg({'left response': lambda x: np.mean((x=='L').astype(float))})
    bias.plot(ax=ax[s],color='green')
    legend(loc=3)

# Plot number of trials and feeds for past week

In [ ]:
mplib_colors = ['b','g','r', 'c', 'm', 'y']
figsize(16.0, 4.0)

for subj,df in behav_data.items():
    data_to_analyze = df[df.index>(dt.datetime.today()-dt.timedelta(weeks=1))]
    if not data_to_analyze.empty:
        blocked = data_to_analyze.groupby(lambda x: (x.date()-dt.datetime.now().date()).days)
        
        days = np.sort(blocked.groups.keys())
        trials_per_day = blocked['response'].count().values
        
        line_color = mplib_colors.pop(0)
        
        line = plot(days, trials_per_day, label=subj + ' trials per day', c=line_color)
        
        if len(days) == 1:
            plot(0, trials_per_day[-1], 'o', c=line_color)
        
        aggregated = blocked.agg({'correct': lambda x: np.mean(x.astype(float)),
                                  'response': lambda x: np.mean((x=='L').astype(float)),
                                  'type_': lambda x: np.sum((x=='normal').astype(float)),
                                  'reward': lambda x: np.sum((x==True).astype(float)),
                                    })
        
        aggregated['reward'].plot(ax=gca(), label=subj + ' feeds per day', ls='--', c=line_color)
        if len(days) == 1:
            plot(0, aggregated['reward'][0], 'o', c=line_color)

title('trials and feeds per day')
h = legend(loc='upper left')

